In [75]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import Ridge

INPUT_FIELDS = [
    'CRITICALITY',
    'CONNECTIVITY',
    'RELIABILITY',
    'IMPORTANCE',
    'PROVIDED_INTERFACE',
    'REQUIRED_INTERFACE',
    'ADT',
    ' PMax',
    'alpha',
    'In Use REPLICA',
    'LOAD',
]
OUTPUT_FIELD = 'OPTIMAL_UTILITY'

data = pd.read_csv('../data/Traning_mRubis_Batch_1.csv')[INPUT_FIELDS + [OUTPUT_FIELD]]
data['Linear'] = data['RELIABILITY'] * data['CRITICALITY'] * (data['PROVIDED_INTERFACE'] + data['REQUIRED_INTERFACE'])
data['Saturating'] = data['RELIABILITY'] * data['CRITICALITY'] * data[' PMax'] * np.tanh(data['alpha'] * data['In Use REPLICA'] / data['LOAD']) * (data['PROVIDED_INTERFACE'] + data['REQUIRED_INTERFACE'])
# NaNs are dropped here, because 'Saturating' might be NaN for rows with 'LOAD' == 0.0
data.dropna(inplace=True)

X = data[INPUT_FIELDS + ['Linear', 'Saturating']]
y = data[OUTPUT_FIELD]

X_linear = X.drop('Saturating', axis=1)
X_saturating = X.drop('Linear', axis=1)

In [76]:
X_linear

,CRITICALITY,CONNECTIVITY,RELIABILITY,IMPORTANCE,PROVIDED_INTERFACE,REQUIRED_INTERFACE,ADT,PMax,alpha,In Use REPLICA,LOAD,Linear
0,25.0,4.0,0.5,22.0,0.0,4.0,1.077155,13.448238,126.901175,12.0,373.0,50.0
1,24.0,2.0,0.5,22.0,1.0,1.0,1.175642,5.323259,154.197090,7.0,4586.0,24.0
2,11.0,3.0,0.5,22.0,1.0,2.0,1.203442,13.462900,85.652412,18.0,377.0,16.5
3,27.0,5.0,0.5,22.0,0.0,5.0,1.356134,8.060657,73.959463,14.0,241.0,67.5
4,27.0,15.0,0.5,22.0,14.0,1.0,1.101619,6.164631,85.670351,24.0,500.0,202.5
...,...,...,...,...,...,...,...,...,...,...,...,...
191,23.0,4.0,0.5,22.0,0.0,4.0,1.296817,12.901919,123.488873,45.0,4586.0,46.0
194,20.0,9.0,0.5,22.0,8.0,1.0,1.006633,9.717122,137.960688,0.0,4586.0,90.0
196,30.0,4.0,0.5,22.0,0.0,4.0,1.460128,12.903071,167.792201,0.0,4586.0,60.0
197,30.0,4.0,0.5,22.0,0.0,4.0,1.460128,12.903071,167.792201,38.0,4586.0,60.0


In [77]:
X_saturating

,CRITICALITY,CONNECTIVITY,RELIABILITY,IMPORTANCE,PROVIDED_INTERFACE,REQUIRED_INTERFACE,ADT,PMax,alpha,In Use REPLICA,LOAD,Saturating
0,25.0,4.0,0.5,22.0,0.0,4.0,1.077155,13.448238,126.901175,12.0,373.0,672.029563
1,24.0,2.0,0.5,22.0,1.0,1.0,1.175642,5.323259,154.197090,7.0,4586.0,29.526479
2,11.0,3.0,0.5,22.0,1.0,2.0,1.203442,13.462900,85.652412,18.0,377.0,222.013273
3,27.0,5.0,0.5,22.0,0.0,5.0,1.356134,8.060657,73.959463,14.0,241.0,543.892601
4,27.0,15.0,0.5,22.0,14.0,1.0,1.101619,6.164631,85.670351,24.0,500.0,1247.668828
...,...,...,...,...,...,...,...,...,...,...,...,...
191,23.0,4.0,0.5,22.0,0.0,4.0,1.296817,12.901919,123.488873,45.0,4586.0,496.867239
194,20.0,9.0,0.5,22.0,8.0,1.0,1.006633,9.717122,137.960688,0.0,4586.0,0.000000
196,30.0,4.0,0.5,22.0,0.0,4.0,1.460128,12.903071,167.792201,0.0,4586.0,0.000000
197,30.0,4.0,0.5,22.0,0.0,4.0,1.460128,12.903071,167.792201,38.0,4586.0,683.795060


In [70]:
y

0      18480.555310
1       4894.866505
2       9762.030924
3      14350.529590
4      51254.059050
           ...     
191    13779.400630
194    29132.737840
196    18789.762880
197    18789.762880
199    29132.737840
Name: OPTIMAL_UTILITY, Length: 91, dtype: float64

In [94]:
errors = []
for train_index, test_index in LeaveOneOut().split(X_linear, y):
    X_train, X_test = X_linear.values[train_index], X_linear.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    errors.append(ridge.predict(X_test) - y_test)
print(f'Linear model - average error: {np.sum(np.abs(errors)) / len(errors):0.2f}')

Linear model - average error: 6329.44


In [92]:
errors = []
for train_index, test_index in LeaveOneOut().split(X_saturating, y):
    X_train, X_test = X_saturating.values[train_index], X_saturating.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    errors.append(ridge.predict(X_test) - y_test)
print(f'Saturating model - average error: {np.sum(np.abs(errors)) / len(errors):0.2f}')

Saturating model - average error: 6248.62
